# DermLIP 피부 질환 진단 시스템

이 노트북은 DermLIP 모델을 사용하여 피부 사진으로부터 질환을 진단하고, 증상과 영향받는 부위, 설명, 중증도를 제공합니다.

## 모델 정보
- **DermLIP-ViT-B/16**: Vision Transformer 기반 (빠른 속도)
- **DermLIP-PanDerm**: PanDerm 아키텍처 기반 (최고 성능)
- **훈련 데이터**: Derm1M (1,029,761 이미지-텍스트 쌍, 390개 피부 질환)

## 진단 가능한 질환 (46개)

### 피부암 및 전암성 병변
- 악성 흑색종, 기저세포암, 편평세포암, 일광 각화증

### 염증성 피부 질환
- 여드름, 습진/아토피, 건선, 주사, 접촉성 피부염, 편평태선

### 감염성 질환
- 헤르페스, 대상포진, 사마귀, 물사마귀, 진균 감염, 어루러기, 농가진, 봉와직염, 옴

### 알레르기 및 면역 질환
- 두드러기, 혈관부종, 약물 발진, 장미색 비강진, 홍반성 루푸스, 경피증, 피부근염, 천포창, 수포성 유천포창, 다형홍반

### 색소 질환
- 백반증, 기미, 흑자, 모반

### 양성 종양 및 성장물
- 지루성 각화증, 모공각화증, 체리 혈관종, 연성 섬유종, 지방종, 화농육아종, 포도주색 반점, 혈관종

### 흉터 및 피부 변화
- 켈로이드, 비후성 반흔

### 탈모 질환
- 원형 탈모, 남성형/여성형 탈모, 휴지기 탈모

## 1. 라이브러리 설치 및 임포트

In [ ]:
# 필요한 라이브러리 설치 (처음 실행시)
!pip install torch torchvision open_clip_torch pillow matplotlib seaborn

In [ ]:
import torch
import torch.nn.functional as F
import open_clip
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# GPU 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'사용 디바이스: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

## 2. 피부 질환 데이터베이스

In [ ]:
# dermlip_diagnosis.py 파일에서 데이터베이스 import
from dermlip_diagnosis import SKIN_DISEASE_DATABASE

print(f'등록된 질환 수: {len(SKIN_DISEASE_DATABASE)}개')
print('\n질환 목록 (카테고리별):')
print('\n📌 피부암 및 전암성 병변:')
for key in ['melanoma', 'basal_cell_carcinoma', 'squamous_cell_carcinoma', 'actinic_keratosis']:
    info = SKIN_DISEASE_DATABASE[key]
    print(f'   • {info["name_ko"]:25s} ({info["name_en"]})')

print('\n📌 염증성 피부 질환:')
for key in ['acne', 'eczema', 'psoriasis', 'rosacea', 'dermatitis', 'lichen_planus']:
    info = SKIN_DISEASE_DATABASE[key]
    print(f'   • {info["name_ko"]:25s} ({info["name_en"]})')

print('\n📌 감염성 질환:')
for key in ['herpes', 'herpes_zoster', 'warts', 'fungal_infection', 'impetigo', 'cellulitis', 'pityriasis_versicolor', 'molluscum_contagiosum', 'scabies']:
    info = SKIN_DISEASE_DATABASE[key]
    print(f'   • {info["name_ko"]:25s} ({info["name_en"]})')

print('\n📌 알레르기 및 면역 질환:')
for key in ['urticaria', 'angioedema', 'drug_eruption', 'pityriasis_rosea', 'lupus_erythematosus', 'scleroderma', 'dermatomyositis', 'pemphigus', 'bullous_pemphigoid', 'erythema_multiforme']:
    info = SKIN_DISEASE_DATABASE[key]
    print(f'   • {info["name_ko"]:25s} ({info["name_en"]})')

print('\n📌 색소 질환:')
for key in ['vitiligo', 'melasma', 'lentigo', 'nevus']:
    info = SKIN_DISEASE_DATABASE[key]
    print(f'   • {info["name_ko"]:25s} ({info["name_en"]})')

print('\n📌 양성 종양 및 성장물:')
for key in ['seborrheic_keratosis', 'keratosis_pilaris', 'cherry_angioma', 'skin_tag', 'lipoma', 'pyogenic_granuloma', 'port_wine_stain', 'hemangioma']:
    info = SKIN_DISEASE_DATABASE[key]
    print(f'   • {info["name_ko"]:25s} ({info["name_en"]})')

print('\n📌 흉터 및 피부 변화:')
for key in ['keloid', 'hypertrophic_scar']:
    info = SKIN_DISEASE_DATABASE[key]
    print(f'   • {info["name_ko"]:25s} ({info["name_en"]})')

print('\n📌 탈모 질환:')
for key in ['alopecia_areata', 'androgenetic_alopecia', 'telogen_effluvium']:
    info = SKIN_DISEASE_DATABASE[key]
    print(f'   • {info["name_ko"]:25s} ({info["name_en"]})')

## 3. DermLIP 진단 시스템 클래스

In [ ]:
class DermLIPDiagnosisSystem:
    """DermLIP 기반 피부 질환 진단 시스템"""

    def __init__(self, model_name='hf-hub:redlessone/DermLIP_ViT-B-16', device='cuda'):
        self.device = device
        self.model_name = model_name

        print(f'모델 로드 중: {model_name}...')
        
        # 모델 로드
        self.model, _, self.preprocess = open_clip.create_model_and_transforms(
            model_name=model_name,
            device=device
        )
        self.tokenizer = open_clip.get_tokenizer(model_name)
        self.model.eval()

        # 클래스 정보 생성
        self.disease_keys = list(SKIN_DISEASE_DATABASE.keys())
        self.class_descriptions = self._generate_class_descriptions()

        print(f'✓ 모델 로드 완료!')
        print(f'✓ {len(self.disease_keys)}개 피부 질환 진단 가능')

    def _generate_class_descriptions(self):
        """DermLIP 모델용 클래스 설명 생성"""
        descriptions = []
        for key in self.disease_keys:
            info = SKIN_DISEASE_DATABASE[key]
            desc = f"a clinical dermatological photograph of {info['name_en']}"
            descriptions.append(desc)
        return descriptions

    def diagnose(self, image_path, top_k=3):
        """피부 질환 진단 수행"""
        print(f'\n이미지 분석 중: {image_path}')

        # 이미지 로드 및 전처리
        image = Image.open(image_path).convert('RGB')
        image_tensor = self.preprocess(image).unsqueeze(0).to(self.device)

        # 이미지 인코딩
        with torch.no_grad():
            image_features = self.model.encode_image(image_tensor)
            image_features = F.normalize(image_features, dim=-1)

        # 텍스트 인코딩
        text_tokens = self.tokenizer(self.class_descriptions).to(self.device)
        with torch.no_grad():
            text_features = self.model.encode_text(text_tokens)
            text_features = F.normalize(text_features, dim=-1)

        # 유사도 계산
        similarity = (image_features @ text_features.T).squeeze(0)
        probabilities = F.softmax(similarity * 100, dim=0).cpu().numpy()

        # 상위 k개 결과
        top_indices = np.argsort(probabilities)[::-1][:top_k]

        diagnosis_results = []
        for rank, idx in enumerate(top_indices, 1):
            disease_key = self.disease_keys[idx]
            disease_info = SKIN_DISEASE_DATABASE[disease_key]
            confidence = float(probabilities[idx]) * 100

            diagnosis_results.append({
                'rank': rank,
                'confidence': confidence,
                'disease_key': disease_key,
                'disease_info': disease_info
            })

        return diagnosis_results, image

    def display_diagnosis(self, diagnosis_results, image):
        """진단 결과 시각화"""
        # 텍스트 결과 출력
        print('\n' + '='*70)
        print('🏥 진단 결과')
        print('='*70)
        
        for result in diagnosis_results:
            rank = result['rank']
            confidence = result['confidence']
            info = result['disease_info']

            print(f'\n[{rank}위] {info["name_ko"]} ({info["name_en"]})')
            print(f'신뢰도: {confidence:.1f}%')
            print('\n📍 주로 영향받는 부위:')
            for area in info['affected_areas']:
                print(f'   • {area}')
            print('\n🔍 주요 증상:')
            for symptom in info['symptoms']:
                print(f'   • {symptom}')
            print(f'\n📋 설명:')
            print(f'   {info["description"]}')
            print(f'\n📊 중증도 분류:')
            for severity in info['severity_levels']:
                print(f'   • {severity}')
            
            if rank < len(diagnosis_results):
                print('\n' + '-'*70)
        
        print('\n' + '='*70)
        print('⚠️  주의: 이 결과는 AI 기반 예측이며, 정확한 진단은 전문의와 상담하세요.')
        print('='*70 + '\n')
        
        # 시각화
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        # 원본 이미지
        axes[0].imshow(image)
        axes[0].axis('off')
        axes[0].set_title('입력 이미지', fontsize=14, fontweight='bold')
        
        # 진단 결과 막대 그래프
        diseases = [f"{r['rank']}. {r['disease_info']['name_ko']}" for r in diagnosis_results]
        confidences = [r['confidence'] for r in diagnosis_results]
        
        colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(diseases)))
        bars = axes[1].barh(range(len(diseases)), confidences, color=colors, alpha=0.8)
        axes[1].set_yticks(range(len(diseases)))
        axes[1].set_yticklabels(diseases)
        axes[1].set_xlabel('신뢰도 (%)', fontsize=12)
        axes[1].set_title('진단 결과', fontsize=14, fontweight='bold')
        axes[1].set_xlim(0, 100)
        axes[1].grid(axis='x', alpha=0.3)
        
        for i, (bar, conf) in enumerate(zip(bars, confidences)):
            axes[1].text(conf + 1.5, i, f'{conf:.1f}%', va='center', fontweight='bold')
        
        plt.tight_layout()
        plt.show()

## 4. 진단 시스템 초기화

In [ ]:
# 진단 시스템 초기화
# 옵션 1: ViT-B/16 모델 (빠름)
diagnosis_system = DermLIPDiagnosisSystem(
    model_name='hf-hub:redlessone/DermLIP_ViT-B-16',
    device=device
)

# 옵션 2: PanDerm 모델 (최고 성능, 더 느림)
# diagnosis_system = DermLIPDiagnosisSystem(
#     model_name='hf-hub:redlessone/DermLIP_PanDerm-base-w-PubMed-256',
#     device=device
# )

## 5. 피부 질환 진단 실행

### 사용 방법:
1. 피부 사진 경로를 `image_path` 변수에 입력
2. 셀 실행
3. 진단 결과 확인 (질환명, 증상, 부위, 설명)

In [ ]:
# 진단할 이미지 경로 설정
image_path = 'path/to/your/skin_image.jpg'  # 여기에 실제 이미지 경로 입력

# 상위 몇 개 결과를 볼지 설정 (기본값: 3)
top_k = 3

# 진단 실행
try:
    diagnosis_results, image = diagnosis_system.diagnose(image_path, top_k=top_k)
    diagnosis_system.display_diagnosis(diagnosis_results, image)
except FileNotFoundError:
    print(f'\n❌ 오류: 이미지 파일을 찾을 수 없습니다: {image_path}')
    print('이미지 경로를 확인하고 다시 시도하세요.\n')
except Exception as e:
    print(f'\n❌ 오류 발생: {e}\n')

## 6. 여러 이미지 일괄 진단 (선택사항)

In [ ]:
# 여러 이미지를 한 번에 진단
image_paths = [
    'path/to/image1.jpg',
    'path/to/image2.jpg',
    'path/to/image3.jpg',
]

for img_path in image_paths:
    try:
        print(f'\n{"="*70}')
        print(f'이미지: {img_path}')
        print('='*70)
        
        diagnosis_results, image = diagnosis_system.diagnose(img_path, top_k=3)
        diagnosis_system.display_diagnosis(diagnosis_results, image)
    except FileNotFoundError:
        print(f'⚠️  이미지를 찾을 수 없음: {img_path}\n')
    except Exception as e:
        print(f'⚠️  오류 발생: {e}\n')

## 7. 사용 가이드

### 진단 절차
1. **이미지 준비**: 진단하고 싶은 피부 부위의 선명한 사진을 준비합니다.
2. **경로 설정**: 5번 셀에서 `image_path` 변수에 이미지 경로를 입력합니다.
3. **진단 실행**: 셀을 실행하면 AI가 이미지를 분석합니다.
4. **결과 확인**: 상위 3개 진단 결과와 각각의 증상, 부위, 설명을 확인합니다.

### 결과 해석
- **신뢰도**: 높을수록 해당 질환일 가능성이 큼
- **주로 영향받는 부위**: 해당 질환이 일반적으로 발생하는 신체 부위
- **주요 증상**: 특징적인 증상들
- **설명**: 질환에 대한 간단한 설명

### 주의사항
⚠️ **이 도구는 참고용이며, 최종 진단은 반드시 피부과 전문의와 상담하세요.**

### 모델 선택
- **ViT-B/16**: 빠른 속도, 일반적인 사용에 적합
- **PanDerm**: 최고 성능, 더 정확한 진단 필요 시 사용

### 팁
- 선명하고 밝은 환경에서 찍은 사진일수록 정확도가 높습니다.
- 병변 부위가 명확하게 보이도록 촬영합니다.
- 여러 각도에서 찍은 사진을 각각 진단해보는 것도 도움이 됩니다.